# [作業目標]
- 使用 Day 17 剛學到的方法, 對較完整的資料生成離散化特徵
- 觀察上述離散化特徵, 對於目標值的預測有沒有幫助

# [作業重點]
- 仿照 Day 17 的語法, 將年齡資料 ('DAYS_BIRTH' 除以 365) 離散化
- 繪製上述的 "離散化標籤" 與目標值 ('TARGET') 的長條圖

In [2]:
# 載入需要的套件
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# 忽略警告訊息
import warnings
warnings.filterwarnings('ignore')

# 設定 data_path
dir_data = './data/'

### 之前做過的處理

In [3]:
# 讀取資料檔
f_app_train = os.path.join(dir_data, 'application_train.csv')
app_train = pd.read_csv(f_app_train)
app_train.shape

(307511, 122)

In [4]:
# 將只有兩種值的類別型欄位, 做 Label Encoder, 計算相關係數時讓這些欄位可以被包含在內
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

# 檢查每一個 column
for col in app_train:
    if app_train[col].dtype == 'object':
        # 如果只有兩種值的類別型欄位
        if len(list(app_train[col].unique())) <= 2:
            # 就做 Label Encoder, 以加入相關係數檢查
            app_train[col] = le.fit_transform(app_train[col])            
print(app_train.shape)
app_train.head()

(307511, 122)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0,M,0,1,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,0,F,0,0,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,1,M,1,1,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,0,F,0,1,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,0,M,0,1,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# 受雇日數為異常值的資料, 另外設一個欄位記錄, 並將異常的日數轉成空值 (np.nan)
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)

# 出生日數 (DAYS_BIRTH) 取絕對值 
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])


KeyError: 'AMT_APPLICATION'

## 練習時間
參考 Day 17 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [36]:
income_data = app_train[['AMT_INCOME_TOTAL'','AMT_CREDIT','AMT_ANNUITY']] 
income_data['equal_width_AMT_INCOME_TOTAL'] = pd.cut(income_data['AMT_INCOME_TOTAL'],10)
print('equal_width_AMT_INCOME_TOTAL:')
print(income_data['equal_width_AMT_INCOME_TOTAL'].value_counts())

income_data['equal_AMT_CREDIT'] = pd.cut(income_data['AMT_CREDIT'],10)
print('equal_AMT_CREDIT:')
print(income_data['equal_AMT_CREDIT'].value_counts())

income_data['equal_width_AMT_ANNUITY'] = pd.cut(income_data['AMT_ANNUITY'],10)
print('equal_width_AMT_ANNUITY:')
print(income_data['equal_width_AMT_ANNUITY'].value_counts())

SyntaxError: invalid syntax (<ipython-input-36-dcc91c289833>, line 1)

In [37]:
income_data['freq_width_AMT_INCOME_TOTAL'] = pd.qcut(income_data['AMT_INCOME_TOTAL'],10)
print('freq_width_AMT_INCOME_TOTAL:')
print(income_data['freq_width_AMT_INCOME_TOTAL'].value_counts())

income_data['freq_width_AMT_CREDIT'] = pd.qcut(income_data['AMT_CREDIT'],10)
print('freq_width_AMT_CREDIT:')
print(income_data['freq_width_AMT_CREDIT'].value_counts())

income_data['freq_width_AMT_ANNUITY'] = pd.qcut(income_data['AMT_ANNUITY'],10)
print('freq_width_AMT_ANNUITYL:')
print(income_data['freq_width_AMT_ANNUITY'].value_counts())

freq_width_AMT_INCOME_TOTAL:
(112500.0, 135000.0]       48849
(180000.0, 225000.0]       44809
(99000.0, 112500.0]        36907
(25649.999, 81000.0]       33391
(147150.0, 162000.0]       31120
(162000.0, 180000.0]       30704
(81000.0, 99000.0]         30280
(270000.0, 117000000.0]    27161
(225000.0, 270000.0]       19957
(135000.0, 147150.0]        4333
Name: freq_width_AMT_INCOME_TOTAL, dtype: int64
freq_width_AMT_CREDIT:
(604152.0, 755190.0]      34066
(44999.999, 180000.0]     33204
(180000.0, 254700.0]      31721
(432000.0, 513531.0]      30826
(513531.0, 604152.0]      30726
(306306.0, 432000.0]      30707
(1133748.0, 4050000.0]    30593
(755190.0, 900000.0]      29958
(900000.0, 1133748.0]     28319
(254700.0, 306306.0]      27391
Name: freq_width_AMT_CREDIT, dtype: int64
freq_width_AMT_ANNUITYL:
(1615.499, 11074.5]    30880
(37516.5, 45954.0]     30808
(24903.0, 28062.0]     30790
(21865.5, 24903.0]     30772
(14701.5, 18189.0]     30763
(32004.0, 37516.5]     30762
(18189.0,

In [33]:
income_data.head()

,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,equal_width_AMT_INCOME_TOTAL,equal_AMT_CREDIT,equal_width_AMT_ANNUITY,freq_width_AMT_INCOME_TOTAL,freq_width_AMT_CREDIT,freq_width_AMT_ANNUITY
0,202500.0,406597.5,24700.5,"(-91324.35, 11723085.0]","(40995.0, 445500.0]","(1359.09, 27256.5]","(180000.0, 225000.0]","(306306.0, 432000.0]","(21865.5, 24903.0]"
1,270000.0,1293502.5,35698.5,"(-91324.35, 11723085.0]","(1246500.0, 1647000.0]","(27256.5, 52897.5]","(225000.0, 270000.0]","(1133748.0, 4050000.0]","(32004.0, 37516.5]"
2,67500.0,135000.0,6750.0,"(-91324.35, 11723085.0]","(40995.0, 445500.0]","(1359.09, 27256.5]","(25649.999, 81000.0]","(44999.999, 180000.0]","(1615.499, 11074.5]"
3,135000.0,312682.5,29686.5,"(-91324.35, 11723085.0]","(40995.0, 445500.0]","(27256.5, 52897.5]","(112500.0, 135000.0]","(306306.0, 432000.0]","(28062.0, 32004.0]"
4,121500.0,513000.0,21865.5,"(-91324.35, 11723085.0]","(445500.0, 846000.0]","(1359.09, 27256.5]","(112500.0, 135000.0]","(432000.0, 513531.0]","(18189.0, 21865.5]"
